In [19]:
import nltk
import sys 
import random

In [20]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det N | N | PropN | Adj NP | NP PP
    VP -> V | Vbar NP | Vbar NP PP | Adv VP | VP Adv | VP Conj VP | V AdvP
    Vbar -> V
    AdvP -> Adv P
    PP -> P NP
    Det -> 'a' | 'an' | 'the' | 'every'| 'some' | 'any'
    P -> 'with' | 'in' | 'on' | 'to' | 'without' | 'from'
    Conj -> 'and' | 'or' | 'but'


    N -> 'boy' | 'student' | 'girl' | 'class' | 'book' | 'teacher'
    PropN -> 'john' | 'mary'
    Adj -> 'eager' | 'smart'
    V -> 'walks' | 'passed' | 'sees' | 'studies' | 'teaches' | 'saw' 
    Adv -> 'eagerly' | 'well'
    
""")

In [21]:
sentences = [ 'John sees Mary', 'A student walks', 'Some girl sees every boy','Every eager student passed']


In [22]:
sentence = random.choice(sentences)
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                            
tokens = sentence.lower().split()
parse_trees = parser.parse(tokens)
if not parse_trees:
    print("No parse tree found.")
for parse_tree in parse_trees:
    print(parse_tree)

(S (NP (Det a) (N student)) (VP (V walks)))


assignment 2

In [23]:
#pretty_print
sentence = random.choice(sentences)
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                            
tokens = sentence.lower().split()
parse_trees = parser.parse(tokens)
if not parse_trees:
    print("No parse tree found.")
for parse_tree in parse_trees:
    parse_tree.pretty_print()

           S         
      _____|______    
     NP           VP 
  ___|_____       |   
Det        N      V  
 |         |      |   
 a      student walks



In [24]:
#practice iterating
for subtree in parse_tree.subtrees(): 
    print(subtree)
print('\n' )
for pos in parse_tree.treepositions(): 
    print(pos)
print('\n')
for pos in parse_tree.treepositions(order="leaves"):
    print(pos)
print('\n')
for pos in parse_tree.treepositions(order="postorder"):
  print(pos)
print('\n')

# Define a function that takes a parse tree, and returns the position of the verb (i.e., a tuple).
#  If your grammar follows the textbook closely, then the position of the verb will depend on
#  whether it is a transitive or intransitive verb (i.e., depending on the presence/absence of a Vbar node).

def get_verb_tuple(tree):
    for subtree in tree.subtrees():
        if  'Vi' or 'Vt' in subtree:
            print(tree.treepositions(subtree))
        else:
            break
print('the verb')
verb = get_verb_tuple(parse_tree)

#Define a function that takes a position in the parse tree (i.e., a tuple like (0, 1, 1)) and returns the 
# position of the parent node ((0, 1) in this case), or None if it has no parent.

def get_parent_node(child_tuple): 
    if len(child_tuple) >= 1: 
        parent_node = child_tuple[:-1]
        return parent_node
    else: 
        return None

#Define a function that takes a parse tree, and returns the position of the subject DP.
def get_position_subj (tree):
    pass

#Define a function that takes a parse tree, and returns the position of the object DP if one exists, and None otherwise
def get_position_obj (tree):
    if  'Vt' in parse_tree.subtrees()[0]: 
        obj =  parse_tree.subtres()[1,1,1]
        return obj
    else: 
        None 

sentence = sentences[0]
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                                  
tokens = sentence.lower().split()
parse_trees = parser.parse(tokens)
if parse_tree in parse_trees:      
    print(parse_tree)
    print(get_position_obj(parse_tree))

IndentationError: unexpected indent (2242872205.py, line 55)

Assignment 3

In [ ]:
translations = { 
    'John sees Mary' : 'SEES(j,m)', 
    'A student walks': 'some x [WALK(x)]', 
    'Some girl sees every boy': 'all y some x [(GIRL(x) ∧ BOY(y)) → SEES(x, y))]',
    'Every eager student passed': }

Assignment 4

In [ ]:
vocabulary = {
    'P': (
        'with',
        'in',
        'on',
        'to',
        'without',
        'from'
    ),
    'N': (
        'boy',
        'student',
        'girl',
        'class',
        'book',
        'teacher'
    ),
    'PropN': (
        'john',
        'mary'
    ),
    'Vi': (
        'walks',
        'passed'
    ),
    'Vt': (
        'sees',
        'teaches'
    ),
    'Adj': (
        'eager',
        'smart'
    ),
    'Adv': (
        'eagerly',
        'well'
    ),
    'Det': (
        'a',
        'an',
        'the',
        'every',
        'some',
        'any'
    ),
    'Conj': (
        'and',
        'or',
        'but'
    )
}



Assignment 5

In [ ]:
# this class would create an object storing both the representation of an element, and its typing information
class Formalization:
    def __init__(self, formula, type_hint, type_=None):
        self.formula = formula
        self.type = type_hint
        self.t = type_
        if type(type_hint) == tuple:
            self.selected = type_hint[0]
            self.returned = type_hint[1]
        else:
            self.selected = None

    def application(self, argument):
        if argument.type == self.selected:
            resulting_formula = self.formula(argument.formula)
            return Formalization(resulting_formula, self.returned)


# this dictionary contains functions to generate lexical entries for different parts of speech.
formalizations = {
    'PropN': lambda name: Formalization(
        name,
        'e', 'PropN'
    ),
    'N': lambda noun: Formalization(
        lambda x: f'{noun.upper()}({x})',
        ('e', 't'), 'N'
    ),
    'Vi': lambda verb: Formalization(
        lambda x: f'{verb.upper()}({x})',
        ('e', 't'), 'Vi'
    ),
    'Vt': lambda verb: Formalization(
        lambda y: lambda x: f'{verb.upper()}({x}, {y})',
        ('e', ('e', 't')), 'Vt'
    ),
    'Adj': lambda adjective: Formalization(
        lambda P: lambda x: f'{adjective.upper()}({x}) ^ {P(x)}',
        (('e', 't'), ('e', 't')), 'Adj'
    ),
    'Adv': lambda adverb: Formalization(
        lambda P: lambda x: f'{adverb.upper()}({P(x)})',
        (('e', 't'), ('e', 't')), 'Adv'
    ),
    'P': lambda preposition: Formalization(
        lambda x: lambda y: f'{preposition.upper()}({x}, {y})',
        ('e', ('e', 't')), 'P'
    ),
    'existential': lambda _: Formalization(
        lambda P: lambda Q: f'Exists x[{P("x")} ^ {Q("x")}]',
        (('e', 't'), (('e', 't'), 't')), 'existential'
    ),
    'universal': lambda _: Formalization(
        lambda P: lambda Q: f'All x[{P("x")} -> {Q("x")}]',
        (('e', 't'), (('e', 't'), 't')), 'universal'
    )
}


def generate_lexicon(vocab, translations):
    lexicon = {}
    for part_of_speech, entries in vocab.items():
        if part_of_speech in translations.keys():
            lexicon.update({part_of_speech: {word: translations[part_of_speech](word) for word in entries}})
        elif type(entries) == dict:
            lexicon[part_of_speech] = {}
            for subtype, words in entries.items():
                lexicon[part_of_speech].update({word: translations[subtype](word) for word in words})
        else:
            lexicon.update({part_of_speech: {word: Formalization(word, 'e') for word in entries}})
    return lexicon


def lexicon_to_terminals(lexicon):
    terminal_rules = ''
    for part_of_speech in lexicon.keys():
        terminals_string = " | ".join(["'" + word + "'" for word in lexicon[part_of_speech].keys()])
        terminal_rules += (part_of_speech + ' -> ' + terminals_string + '\n\t')
    return terminal_rules


extensional_lexicon = generate_lexicon(vocabulary, formalizations)

terminals_entries = lexicon_to_terminals(extensional_lexicon)

extensional_grammar = f"""
    S -> NP VP
    NP -> Det Nom | PropN
    Nom -> N  | Adj Nom | Nom PP
    VP -> Vi | Vt  | Vbar NP | Vbar NP PP | Adv VP | VP Adv | VP Conj VP | V AdvP
    Vbar -> Vi | Vt
    AdvP -> Adv P
    PP -> P NP
    {terminals_entries}
"""

NameError: name 'vocabulary' is not defined

Assignment 6

In [ ]:
existentials = ['a', 'some', 'an']
universal = ['every']
def translate_to_logic(tree = nltk.Tree):
    result = {}
    prev_was_leave = False
    leaf = ''
    root_cause = None
    for node in tree.treepositions(order='postorder'):
        if isinstance(tree[node], nltk.Tree):
            if len(tree[node]) > 1:
                if tree[node].label() == 'S':
                    if result[node + (1,)].t == 'existential' or result[node + (1,)].t =='universal': #checks if the vp has an quantifier
                        result[node] = result[node + (0,)].application(result[node + (1,)]) #it will apple the r child to the left 
                    else:
                        result[node] = result[node + (1,)].application(result[node + (0,)])
                else:
                    result[node] = result[node + (0,)].application(result[node + (1,)])
            else:
                if prev_was_leave:
                    if leaf in existentials:
                        applier = 'existential'
                    elif leaf in universal:
                        applier = 'universal'
                    else:
                        applier = tree[node].label()
                    result[node] = formalizations[applier](leaf)
                    root_cause = result[node]
                    prev_was_leave = False
                else:
                    result[node] = root_cause
        else:
            prev_was_leave = True
            leaf = tree[node]
    return result
sentence = sentences[0]
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                                  
tokens = sentence.lower().split()
parse_tree = list(parser.parse(tokens))[0]
parse_tree.pretty_print()
res = translate_to_logic(parse_tree)
res[()].formula

NameError: name 'nltk' is not defined